In [1]:
from sentence_transformers import SentenceTransformer

In [2]:
from ast import literal_eval

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
md = pd.read_csv('./data2/movies_metadata.csv') # dataset do tmdb com os filmes que existem tanto nele, quanto no imdb

C:\Users\joao.franca\AppData\Local\Temp\ipykernel_28792\2562282817.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  md = pd.read_csv('./data2/movies_metadata.csv') # dataset do tmdb com os filmes que existem tanto nele, quanto no imdb


#### Preprocess

In [15]:
md.drop(['adult', 'homepage', 'imdb_id', 'poster_path', 'video'] , axis=1, inplace=True)
md[md['vote_average'].isna() == True]
md.drop(md[md['vote_average'].isna() == True].index, axis=0, inplace=True)
md['belongs_to_collection'] = md['belongs_to_collection'].notna().astype('int')
md['genres'] = md['genres'].apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
md['production_companies'] = md['production_companies'].apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
md['production_countries'] = md['production_countries'].apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
md['spoken_languages'] = md['spoken_languages'].apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

#### Feature Engineer

In [19]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').dt.year.convert_dtypes()
md = md[md['status'] == 'Released']
del md['status']

In [21]:
m = round(md['vote_count'].quantile(0.95))  # dica: pegar so filmes com vote_count quantile > 95 (so filmes mainstream)
C = round(md['vote_average'].mean(), 1)     # isso fara nosso sistema de rec recomendar só filme mainstream tbm
    
df = md[(md['vote_count'] >= m)][['id', 'title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres', 'overview']]

df.shape

(2255, 8)

In [25]:
# features para ajudar > recomendacoes baseadas em keywords e elenco (creditos) daquele filme
credits = pd.read_csv('./data2/credits.csv')
keywords = pd.read_csv('./data2/keywords.csv')

credits.drop_duplicates(inplace = True)
keywords.drop_duplicates(inplace = True)

In [27]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
df['id'] = df['id'].astype('int')

In [29]:
df = df.merge(credits, on='id')
df = df.merge(keywords, on='id')

In [31]:
df['cast'] = df['cast'].apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df['cast'] = df['cast'].apply(lambda x: x[:5] if len(x) >=5 else x)

In [33]:
df['keywords'] = df['keywords'].apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [35]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

df['director'] = df['crew'].apply(literal_eval).apply(get_director)

In [37]:
# Removing space between names in order to differentiate between different actors and directors

df['cast'] = df['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
df['director'] = df['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))

In [39]:
# Changing the datatype to list to enable concatenation
df['director'] = df['director'].apply(lambda x: [i for i in x.split()])

# Creating a new column by concatenating all the relavant metadata info
df['metadata'] = df['genres'] + df['keywords'] + df['director'] + df['cast']

# Taking all the items in the metadata and joining it into a single string
df['metadata'] = df['metadata'].apply(lambda x: ' '.join(x))

In [79]:
df.to_excel("check.xlsx")

In [41]:
model = SentenceTransformer('all-mpnet-base-v2')

C:\Users\joao.franca\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [45]:
meta_data = np.array(df['metadata'])
embeddings = model.encode(meta_data, show_progress_bar=True)

Batches:   0%|          | 0/71 [00:00<?, ?it/s]

In [47]:
model.get_sentence_features

<bound method SentenceTransformer.get_sentence_features of SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)>

In [49]:
# Almost 1/4th of our data (531/2255) would get truncated as it exceeds the max_seq_length of 384. But it shouldn't affect the performance too much.

i = [len(i) for i in meta_data if len(i) > 384]
len(i)

46

In [51]:
embeddings.shape

(2255, 768)

In [53]:
cos_sim_data = pd.DataFrame(cosine_similarity(embeddings))

In [77]:
cos_sim_data.to_excel("testin.xlsx")

In [55]:
df = df.reset_index()
data = df[['title','genres','year']]
indices = pd.Series(df.index, index=df['title'])

In [57]:
def get_recommendations(title, N = 30):
    idx = indices[title]
    sim_scores = list(enumerate(cos_sim_data[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:N+1]
    movie_indices = [i[0] for i in sim_scores]
    sim_scores = pd.DataFrame(sim_scores, columns = ['index', 'similarity_score']) 
    final_data = data.iloc[movie_indices]
    final_data = final_data.merge(sim_scores, left_index = True, right_on ='index')
    final_data['similarity_score'] = round(final_data['similarity_score']*100,2)
    del final_data['index']
    return final_data

In [75]:
get_recommendations('Taxi Driver', 20)

,title,genres,year,similarity_score
0,Chinatown,"[Crime, Drama, Mystery, Thriller]",1974,82.40
1,Collateral,"[Drama, Crime, Thriller]",2004,81.06
2,Out of the Furnace,"[Thriller, Drama, Crime]",2013,80.74
3,Looper,"[Action, Thriller, Science Fiction]",2012,80.45
4,Payback,"[Drama, Action, Thriller, Crime]",1999,80.43
5,Gone Baby Gone,"[Crime, Drama, Mystery]",2007,80.29
6,The Jacket,"[Drama, Mystery, Thriller, Fantasy]",2005,80.05
7,Psycho,"[Drama, Horror, Thriller]",1960,79.55
8,Cape Fear,"[Crime, Thriller]",1991,79.34
9,Shoot 'Em Up,"[Action, Thriller, Comedy, Crime]",2007,79.21
